## Binary Classification Example
### http://bit.ly/2jWiCQO

In this notebok, we will build a binary classification application using the MLlib Pipelines API (https://spark.apache.org/docs/latest/ml-pipeline.html). 
The Pipelines API provides higher-level API built on top of DataFrames for constructing ML pipelines. 

#### You can read more about the Pipelines API in the programming guide - http://spark.apache.org/docs/latest/ml-guide.html

Binary Classification is the task of predicting a binary label. 
E.g., is an email spam or not spam? Should I show this ad to this user or not? 
Will it rain tomorrowor not? 
This notebook demonstrates algorithms for making these types of predictions.

## Dataset Review
The Adult dataset we are going to use is publicly available at the UCI Machine Learning Repository. 
This data derives from census data, and consists of information about 48842 individuals and their annual income. 
We will use this information to predict if an individual earns >50k a year or <=50K a year. 
The dataset is rather clean, and consists of both numeric and categorical variables.

## Attribute Information:

- age: continuous
- workclass: Private,Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked
- fnlwgt: continuous
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc...
- education-num: continuous
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent...
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners...
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
- sex: Female, Male
- capital-gain: continuous
- capital-loss: continuous
- hours-per-week: continuous
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany...
- Target/Label: - <=50K, >50K

In [ ]:
import pixiedust

In [ ]:
from pyspark.sql.types import *

schema = StructType([
        StructField("age", DoubleType(), True),
        StructField("workclass", StringType(), True),
        StructField("fnlwgt", DoubleType(), True),
        StructField("education", StringType(), True),
        StructField("education_num", DoubleType(), True),
        StructField("marital_status", StringType(), True),
        StructField("occupation", StringType(), True),
        StructField("relationship", StringType(), True),
        StructField("race", StringType(), True),
        StructField("sex", StringType(), True),
        StructField("capital_gain", DoubleType(), True),
        StructField("capital_loss", DoubleType(), True),
        StructField("hours_per_week", DoubleType(), True),
        StructField("native_country", StringType(), True),
        StructField("income", StringType(), True)        
])

In [ ]:
from pyspark.sql import SparkSession

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_19099026f8df40b6aec4353c7e897e95(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'cc29768790ec45439a43668592b02f84')
    hconf.set(prefix + '.username', '')
    hconf.set(prefix + '.password', '')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_19099026f8df40b6aec4353c7e897e95(name)

spark = SparkSession.builder.getOrCreate()

# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initalized for you.
# The following variable contains the path to your file on your Object Storage.
path_1 = "swift://Databricks." + name + "/adult.data"

In [ ]:
dataset = (spark.read
  .schema(schema)
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')
  .option('header', 'true')
  .load(path_1))
  
# dataset.take(5)
display(dataset)

In [ ]:
dataset.printSchema

In [ ]:
cols = dataset.columns

### Preprocess Data
Since we are going to try algorithms like Logistic Regression, we will have to convert the categorical variables in the dataset into numeric variables. 
There are 2 ways we can do this.

### Category Indexing.
This is basically assigning a numeric value to each category from {0, 1, 2, ...numCategories-1}. 
This introduces an implicit ordering among your categories, and is more suitable for ordinal variables (eg: Poor: 0, Average: 1, Good: 2)

### One-Hot Encoding http://spark.apache.org/docs/latest/ml-features.html#onehotencoder
This converts categories into binary vectors with at most one nonzero value (eg: (Blue: [1, 0]), (Green: [0, 1]), (Red: [0, 0]))

In this dataset, we have ordinal variables like education (Preschool - Doctorate), and also nominal variables like relationship (Wife, Husband, Own-child, etc). 
For simplicity’s sake, we will use One-Hot Encoding to convert all categorical variables into binary vectors. 
It is possible here to improve prediction accuracy by converting each categorical column with an appropriate method.

Here, we will use a combination of StringIndexer (http://spark.apache.org/docs/latest/ml-features.html#stringindexer) and OneHotEncoder to convert the categorical variables. 
The OneHotEncoder will return a SparseVector.

Since we will have more than 1 stages of feature transformations, we use a Pipeline to tie the stages together. 
This simplifies our code.

In [ ]:
###One-Hot Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ["workclass", "education", "marital_status", "occupation", "relationship", "race", "sex", "native_country"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]

The above code basically indexes each categorical column using the StringIndexer, and then converts the indexed categories into one-hot encoded variables. 
The resulting output has the binary vectors appended to the end of each row.

We use the StringIndexer again here to encode our labels to label indices.

In [ ]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "income", outputCol = "label")
stages += [label_stringIdx]

Next, we will use the VectorAssembler (http://spark.apache.org/docs/latest/ml-features.html#vectorassembler) to combine all the feature columns into a single vector column. 
This will include both the numeric columns and the one-hot encoded binary vector columns in our dataset.

In [ ]:
# Transform all features into a vector using VectorAssembler
numericCols = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

We finally run our stages as a Pipeline. 
This puts the data through all of the feature transformations we described in a single call.

In [ ]:
# Create a Pipeline. https://spark.apache.org/docs/latest/ml-pipeline.html
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(dataset)
dataset = pipelineModel.transform(dataset)

# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)
display(dataset)
#dataset.take(5)

In [ ]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print trainingData.count()
print testData.count()

### Fit and Evaluate Models
We are now ready to try out some of the Binary Classification algorithms available in the Pipelines API.

Out of these algorithms, the below are also capable of supporting multiclass classification with the Python API: 
    - Decision Tree Classifier (http://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier)
    - Random Forest Classifier (http://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier)

These are the general steps we will take to build our models: 
    - Create initial model using the training set 
    - Tune parameters with a ParamGrid and 5-fold Cross Validation 
    - Evaluate the best model obtained from the Cross Validation using the test set

We will be using the BinaryClassificationEvaluator to evaluate our models. 
    - The default metric used here is areaUnderROC.

### Logistic Regression
You can read more about Logistic Regression from the Programming Guide here http://spark.apache.org/docs/latest/ml-classification-regression.html#logistic-regression. 
In the Pipelines API, we are now able to perform Elastic-Net Regularization with Logistic Regression, as well as other linear methods.

Note: As of Spark 2.0.0, The Python API does not yet support multiclass classification for Logistic Regression, but will be available in future.

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [ ]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)

In [ ]:
predictions.printSchema()

In [ ]:
# View model's predictions and probabilities of each prediction class
# You can select any columns in the above schema to view as well. For example's sake we will choose age & occupation
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
display(selected)
#selected.take(5)

We can make use of the BinaryClassificationEvaluator method to evaluate our model. 
The Evaluator expects two input columns: (rawPrediction, label).

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

Note that the default metric for the BinaryClassificationEvaluator is areaUnderROC

In [ ]:
evaluator.getMetricName()

The evaluator currently accepts 2 kinds of metrics - areaUnderROC and areaUnderPR. We can set it to areaUnderPR by using evaluator.setMetricName(“areaUnderPR”).

Now we will try tuning the model with the ParamGridBuilder and the CrossValidator.

If you are unsure what params are available for tuning, you can use explainParams() to print a list of all params and their definitions.

In [ ]:
print lr.explainParams()

As we indicate 3 values for regParam, 3 values for maxIter, and 2 values for elasticNetParam, this grid will have 3 x 3 x 3 = 27 parameter settings for CrossValidator to choose from. We will create a 5-fold cross validator.

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [ ]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [ ]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [ ]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

We can also access the model’s feature weights and intercepts easily

In [ ]:
print 'Model Intercept: ', cvModel.bestModel.intercept

In [ ]:
# weights = cvModel.bestModel.weights
# on Spark 2.X weights are available as ceofficients
weights = cvModel.bestModel.coefficients
weights = map(lambda w: (float(w),), weights)  # convert numpy type to float, and to tuple
weightsDF = sqlContext.createDataFrame(weights, ["Feature Weight"])
display(weightsDF)
#weightsDF.take(5)

In [ ]:
# View best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
display(selected)
#selected.take(5)

# Decision Trees

You can read more about Decision Trees in the Spark MLLib Programming Guide here (http://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier)

The Decision Trees algorithm is popular because it handles categorical data and works out of the box with multiclass classification tasks.

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)

# Train model with Training Data
dtModel = dt.fit(trainingData)

We can extract the number of nodes in our decision tree as well as the tree depth of our model.

In [ ]:
print "numNodes = ", dtModel.numNodes
print "depth = ", dtModel.depth

In [ ]:
# Make predictions on test data using the Transformer.transform() method.
predictions = dtModel.transform(testData)

In [ ]:
predictions.printSchema()

In [ ]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
display(selected)
#selected.take(5)

We will evaluate our Decision Tree model with BinaryClassificationEvaluator.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

Entropy and the Gini coefficient are the supported measures of impurity for Decision Trees. This is Gini by default. Changing this value is simple, model.setImpurity("Entropy").

In [ ]:
dt.getImpurity()

Now we will try tuning the model with the ParamGridBuilder and the CrossValidator.

As we indicate 3 values for maxDepth and 3 values for maxBin, this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from. We will create a 5-fold CrossValidator.

In [ ]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(dt.maxDepth, [1,2,6,10])
             .addGrid(dt.maxBins, [20,40,80])
             .build())

In [ ]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# Takes ~5 minutes

In [ ]:
print "numNodes = ", cvModel.bestModel.numNodes
print "depth = ", cvModel.bestModel.depth

In [ ]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [ ]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

In [ ]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
display(selected)
#selected.take(5)

# Random Forest

Random Forests uses an ensemble of trees to improve model accuracy.

You can read more about Random Forest from the programming guide here (http://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-regression)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [ ]:
# Make predictions on test data using the Transformer.transform() method.
predictions = rfModel.transform(testData)

In [ ]:
predictions.printSchema()

In [ ]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
display(selected)

We will evaluate our Random Forest model with BinaryClassificationEvaluator.

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

Now we will try tuning the model with the ParamGridBuilder and the CrossValidator.

As we indicate 3 values for maxDepth, 2 values for maxBin, and 2 values for numTrees, this grid will have 3 x 2 x 2 = 12 parameter settings for CrossValidator to choose from. We will create a 5-fold CrossValidator.

In [ ]:
# Create ParamGrid for Cross Validation
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 4, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

In [ ]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations.  This can take about 6 minutes since it is training over 20 trees!
cvModel = cv.fit(trainingData)

In [ ]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

In [ ]:
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

In [ ]:
# View Best model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "age", "occupation")
display(selected)

# Make Predictions

As Random Forest gives us the best areaUnderROC value, we will use the bestModel obtained from Random Forest for deployment, and use it to generate predictions on new data. In this example, we will simulate this by generating predictions on the entire dataset.

In [ ]:
bestModel = cvModel.bestModel

In [ ]:
# Generate predictions for entire dataset
finalPredictions = bestModel.transform(dataset)

In [ ]:
# Evaluate best model
evaluator.evaluate(finalPredictions)

In this example, we will also look into predictions grouped by age and occupation.

In [ ]:
finalPredictions.createOrReplaceTempView("finalPredictions")

In [ ]:
#https://spark.apache.org/docs/2.0.0-preview/sql-programming-guide.html#sql

In [ ]:
display(spark.sql("select * FROM finalPredictions"))

#### In an operational environment, analysts may use a similar machine learning pipeline to obtain predictions on new data, organize it into a table and use it for analysis or lead targeting.

In [ ]:
# http://cdn2.hubspot.net/hubfs/438089/notebooks/spark2.0/SparkSession.html

In [ ]:
display(spark.sql("SELECT occupation, prediction, count(*) as count FROM finalPredictions GROUP BY occupation, prediction ORDER BY occupation"))

In [ ]:
display(spark.sql("SELECT age, prediction, count(*) AS count FROM finalPredictions GROUP BY age, prediction ORDER BY age"))

In [ ]:
display(sqlContext.sql("SELECT age, prediction, count(*) AS count FROM finalPredictions GROUP BY age, prediction ORDER BY age"))

In [ ]:
#https://jaceklaskowski.gitbooks.io/mastering-apache-spark/content/spark-sql-Catalog.html

In [ ]:
spark.sql("SHOW TABLES").show()

In [ ]:
spark.catalog.listTables()